In [4]:
#importing all the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder


In [8]:
#reading the data
traindf=pd.read_csv("./data/train.csv")
print(traindf.head())

C:\Users\krish\AppData\Local\Temp\ipykernel_16564\910084911.py:2: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  traindf=pd.read_csv("./data/train.csv")


   resultId  racerId  driverId  constructorId number  grid position_x  \
0     20025      833       642             51      2     1          1   
1     20025      833       642             51      2     1          1   
2     20025      833       642             51      2     1          1   
3     20025      833       642             51      2     1          1   
4     20025      833       642             51      2     1          1   

  positionText_x  positionOrder  points  ...  points_y position  \
0              1              1     9.0  ...      3.33        3   
1              1              1     9.0  ...      6.33        3   
2              1              1     9.0  ...      6.33        4   
3              1              1     9.0  ...     10.33        3   
4              1              1     9.0  ...     10.33        4   

  positionText_y wins constructorRef     company nationality_y  \
0              3    0           alfa  Alfa Romeo         Swiss   
1              3    0     

In [27]:
# replacing all /N values by Nan
traindf.replace('\\N', None, inplace=True)

# dropping values that are not neeeded
df=traindf.drop(['time_x', 'timetaken_in_millisec', 'fastestLap', 'rank', 'fastestLapTime', 'max_speed', 'time_y', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date'
            , 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'driver_num', 'driver_code', 'url_y'
             ,"number", "position_x", 'nationality_y', 'url', 'status', 'positionText_y', 'constructorRef', 'forename', 'surname', 'driverRef', 'positionText_x', 'nationality'], axis=1)


#converting the dob and date column into years to determine the age of the driver
df['dob'] = pd.to_datetime(df['dob'])
df['date'] = pd.to_datetime(df['date'])
df['time_difference'] = df['date'] - df['dob']
df['years'] = (df['time_difference'].dt.days / 365.25).astype(int)

df = df.drop(['dob', 'date', 'time_difference'], axis=1)

# using frequency encoding for grand_prix and company
# frequency_grand_prix = df['grand_prix'].value_counts(normalize=True)
# df['grand_prix_encoded'] = df['grand_prix'].map(frequency_grand_prix)

frequency_company = df['company'].value_counts(normalize=True)
df['company_encoded'] = df['company'].map(frequency_company)

df = df.drop(['grand_prix', 'company'], axis=1)


#impute null values
# imputer = SimpleImputer(strategy='mean')  # or 'median'
# df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# df
# print(df.isnull().any())
# print(df.head())
# print(df.dtypes)

In [28]:
df

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,statusId,year,round,circuitId,driverStandingsId,raceId_y,points_y,position,wins,result_driver_standing,years,company_encoded
0,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46079,792,3.33,3,0,922731975,43,0.011521
1,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46101,793,6.33,3,0,923172525,43,0.011521
2,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46134,794,6.33,4,0,923833350,43,0.011521
3,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46202,795,10.33,3,0,925195050,43,0.011521
4,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46271,796,10.33,4,0,926576775,43,0.011521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830096,22109,899,17,9,4,2,18.0,71,1,2013,19,18,18145,16,61.50,4,2,401167805,37,0.026428
2830097,22109,899,17,9,4,2,18.0,71,1,2013,19,18,29874,17,69.50,4,2,660484266,37,0.026428
2830098,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63744,337,4.00,8,0,1409316096,37,0.026428
2830099,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63768,338,6.00,10,0,1409846712,37,0.026428


In [13]:
testdf = pd.read_csv("./data/test.csv")

C:\Users\krish\AppData\Local\Temp\ipykernel_16564\811105806.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  testdf = pd.read_csv("./data/test.csv")


In [32]:
# replacing all /N values by Nan
testdf.replace('\\N', None, inplace=True)

# dropping values that are not neeeded
df1=testdf.drop(['time_x', 'timetaken_in_millisec', 'fastestLap', 'rank', 'fastestLapTime', 'max_speed', 'time_y', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date'
            , 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'driver_num', 'driver_code', 'url_y'
             ,"number", "position_x", 'nationality_y', 'url', 'status', 'constructorRef', 'forename', 'surname', 'driverRef', 'positionText_x', 'nationality'], axis=1)


#converting the dob and date column into years to determine the age of the driver
df1['dob'] = pd.to_datetime(df1['dob'])
df1['date'] = pd.to_datetime(df1['date'])
df1['time_difference'] = df1['date'] - df1['dob']
df1['years'] = (df1['time_difference'].dt.days / 365.25).astype(int)

df1 = df1.drop(['dob', 'date', 'time_difference'], axis=1)

# using frequency encoding for grand_prix and company
# frequency_grand_prix = df1['grand_prix'].value_counts(normalize=True)
# df1['grand_prix_encoded'] = df1['grand_prix'].map(frequency_grand_prix)

frequency_company = df1['company'].value_counts(normalize=True)
df1['company_encoded'] = df1['company'].map(frequency_company)

df1 = df1.drop(['grand_prix', 'company'], axis=1)


#impute null values
# imputer = SimpleImputer(strategy='mean')  # or 'median'
# df_imputed = pd.DataFrame(imputer.fit_transform(df1), columns=df1.columns)

# df


C:\Users\krish\AppData\Local\Temp\ipykernel_16564\2546326430.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['dob'] = pd.to_datetime(df1['dob'])
C:\Users\krish\AppData\Local\Temp\ipykernel_16564\2546326430.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['date'] = pd.to_datetime(df1['date'])


In [33]:
df1

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,statusId,year,round,circuitId,driverStandingsId,raceId_y,points_y,wins,result_driver_standing,years,company_encoded
0,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12474,93,16.0,0,298739826,36,0.08897
1,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12494,94,21.0,0,299218806,36,0.08897
2,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12514,95,21.0,0,299697786,36,0.08897
3,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12534,96,25.0,0,300176766,36,0.08897
4,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12554,97,25.0,0,300655746,36,0.08897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352923,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72085,1106,5.0,0,1880337225,22,0.08897
352924,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72125,1107,5.0,0,1881380625,22,0.08897
352925,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72145,1108,17.0,0,1881902325,22,0.08897
352926,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72165,1109,27.0,0,1882424025,22,0.08897


In [14]:
validationdf = pd.read_csv("./data/validation.csv")

C:\Users\krish\AppData\Local\Temp\ipykernel_16564\3722391647.py:1: DtypeWarning: Columns (13,16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  validationdf = pd.read_csv("./data/validation.csv")


In [34]:
# replacing all /N values by Nan
validationdf.replace('\\N', None, inplace=True)

# dropping values that are not neeeded
df2=validationdf.drop(['time_x', 'timetaken_in_millisec', 'fastestLap', 'rank', 'fastestLapTime', 'max_speed', 'time_y', 'url_x', 'fp1_date', 'fp1_time', 'fp2_date'
            , 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'driver_num', 'driver_code', 'url_y'
             ,"number", "position_x", 'nationality_y', 'url', 'status', 'positionText_y', 'constructorRef', 'forename', 'surname', 'driverRef', 'positionText_x', 'nationality'], axis=1)


#converting the dob and date column into years to determine the age of the driver
df2['dob'] = pd.to_datetime(df2['dob'])
df2['date'] = pd.to_datetime(df2['date'])
df2['time_difference'] = df2['date'] - df2['dob']
df2['years'] = (df2['time_difference'].dt.days / 365.25).astype(int)

df2 = df2.drop(['dob', 'date', 'time_difference'], axis=1)

# using frequency encoding for grand_prix and company
# frequency_grand_prix = df2['grand_prix'].value_counts(normalize=True)
# df2['grand_prix_encoded'] = df2['grand_prix'].map(frequency_grand_prix)

frequency_company = df2['company'].value_counts(normalize=True)
df2['company_encoded'] = df2['company'].map(frequency_company)

df2 = df2.drop(['grand_prix', 'company'], axis=1)


#impute null values
# imputer = SimpleImputer(strategy='mean')  # or 'median'
# df_imputed = pd.DataFrame(imputer.fit_transform(df1), columns=df1.columns)

# df


In [35]:
df2

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,statusId,year,round,circuitId,driverStandingsId,raceId_y,points_y,position,wins,result_driver_standing,years,company_encoded
0,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63816,340,28.0,8,0,1410907944,37,0.104177
1,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63840,341,53.0,4,1,1411438560,37,0.104177
2,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63888,342,78.0,1,2,1412499792,37,0.104177
3,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63912,343,93.0,1,2,1413030408,37,0.104177
4,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63936,344,103.0,3,2,1413561024,37,0.104177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353757,23949,997,4,1,13,8,4.0,70,11,2018,9,70,71200,88,123.0,1,6,1705168800,36,0.139328
353758,23949,997,4,1,13,8,4.0,70,11,2018,9,70,71227,89,133.0,1,7,1705815423,36,0.139328
353759,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12424,90,6.0,3,0,297542376,36,0.139328
353760,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12438,91,8.0,5,0,297877662,36,0.139328


In [23]:
df1

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,statusId,year,round,circuitId,driverStandingsId,raceId_y,points_y,position,wins,result_driver_standing,years,company_encoded
0,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46079,792,3.33,3,0,922731975,43,0.011521
1,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46101,793,6.33,3,0,923172525,43,0.011521
2,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46134,794,6.33,4,0,923833350,43,0.011521
3,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46202,795,10.33,3,0,925195050,43,0.011521
4,20025,833,642,51,1,1,9.0,70,1,1950,1,9,46271,796,10.33,4,0,926576775,43,0.011521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830096,22109,899,17,9,4,2,18.0,71,1,2013,19,18,18145,16,61.50,4,2,401167805,37,0.026428
2830097,22109,899,17,9,4,2,18.0,71,1,2013,19,18,29874,17,69.50,4,2,660484266,37,0.026428
2830098,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63744,337,4.00,8,0,1409316096,37,0.026428
2830099,22109,899,17,9,4,2,18.0,71,1,2013,19,18,63768,338,6.00,10,0,1409846712,37,0.026428


In [25]:
params = {
    'objective': 'regression',
    'metric': 'rmse',  # RMSE for regression
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'device': 'gpu',  # Use GPU for training
    'gpu_platform_id': 0,  # Specify GPU platform ID (0 by default)
    'gpu_device_id': 0  # Specify GPU device ID (0 by default)
}

In [36]:
X_train = df.drop(columns=['position'])
y_train = df['position']

X_test = df2.drop(columns=['position'])
y_test = df2['position']

predictkardo = df1
predictkardo

,resultId,racerId,driverId,constructorId,grid,positionOrder,points,laps,statusId,year,round,circuitId,driverStandingsId,raceId_y,points_y,wins,result_driver_standing,years,company_encoded
0,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12474,93,16.0,0,298739826,36,0.08897
1,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12494,94,21.0,0,299218806,36,0.08897
2,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12514,95,21.0,0,299697786,36,0.08897
3,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12534,96,25.0,0,300176766,36,0.08897
4,23949,997,4,1,13,8,4.0,70,11,2018,9,70,12554,97,25.0,0,300655746,36,0.08897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352923,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72085,1106,5.0,0,1880337225,22,0.08897
352924,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72125,1107,5.0,0,1881380625,22,0.08897
352925,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72145,1108,17.0,0,1881902325,22,0.08897
352926,26085,1110,857,1,5,20,0.0,0,130,2023,12,13,72165,1109,27.0,0,1882424025,22,0.08897


In [37]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Train the model
num_round = 5000
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2684
[LightGBM] [Info] Number of data points in the train set: 2830101, number of used features: 19
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (53.98 MB) transferred to GPU in 0.100398 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 12.332001
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (43.20 MB) transferred to GPU in 0.048792 secs. 1 sparse feature groups
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (43.18 MB) transferred to GPU in 0.042586 secs. 1 sparse feature groups
[LightGBM] [Info] Size of histogram

In [40]:

# Predict on the test set
# y_pred = bst.predict(predictkardo, num_iteration=bst.best_iteration)
y_pred = bst.predict(predictkardo, num_iteration=bst.best_iteration)

# Evaluate the model
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# r2 = r2_score(y_test, y_pred)

In [45]:
# print(f'RMSE: {rmse}')
# print(f'R²: {r2}')

# Optionally, save the model
bst.save_model('lightgbm_model.txt')

# Print sample predictions
predictions_df = predictkardo.copy()
# predictions_df['actual_position'] = y_test
predictions_df['position'] = y_pred
# predictions_df
submissions = predictions_df[['position', 'result_driver_standing']]
print(submissions)
# predictions_df

         position  result_driver_standing
0        3.721235               298739826
1        4.335585               299218806
2        5.740596               299697786
3        4.417762               300176766
4        4.953641               300655746
...           ...                     ...
352923  14.595312              1880337225
352924  14.595312              1881380625
352925   8.292086              1881902325
352926   7.602627              1882424025
352927   7.818199              1882971810

[352928 rows x 2 columns]


In [44]:
submissions.to_csv('submission.csv', index=False)